You got a job at Charles Schwab! 

* As your first project, your boss wants you to create an ETF Recommender for your company.

* Whenever an investor is about to buy an ETF, you can recommend him/her other ETFs that are similar.

 * Parse the web for ETF holdings

 * Devise a method to compute the similarity between ETFs

 * Recommend similar ETFs, and state if there are any benefits from switching such as lower expense ratios.

* What would your recommendation be if an investor wishes to buy:
  * SPY
  * QQQ
  * ARKK
  * IBB
  * DIA
  * IWM
  * XLF
  * RTH

* In particular for QQQ, what would your recommendation be? Are investors rational in their behavior, or are there limits to rationality?

### Objective
Obtain the largest ETFs (AUM > $2bn)
in the world by web parsing using
python, retrieving their holdings, and
building a recommendation so that
investors can find a better alternative

In [ ]:
# Import all required libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen
from urllib.request import Request

- Step 1: Set up your database using MongoD
- Step 2: Determine the universe of ETFs
- Step 3: Web Mining(after you downloaded data from your sources, you'll be saving it to MongoDB)
- Step 4: Cosine similarity doesn't care about the magnitutde, u only care about the angle. You create a distance matrix between every pair of ETFs. You would compare it with the expense ratio and make your recommendation. 

### Step 1: Setup MongoDB
- Sign up for an account
- Create a cluster

In [ ]:
! python -m pip install pymongo==3.7. #Installing required libraries
! python -m pip install pymongo[srv]
import pymongo # Importing the libraries
from pymongo import MongoClient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.8/626.8 KB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymongo: filename=pymongo-3.7.0-cp38-cp38-linux_x86_64.whl size=436225 sha256=ba624ae4f4b952cb1d6483897becd843f2875c4bff4049d933a65991255cb884
  Stored in directory: /root/.cache/pip/wheels/33/33/8a/e080ffb7c749ca54a191fbf42095b6e4fcb66bd305a3f2b1b5
Successfully built pymongo
  Attempting uninstall: pymongo
    Found existing installation: pymongo 4.3.3
    Uninstalling pymongo-4.3.3:
      Successfully uninstalled pymongo-4.3.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 KB 7.6 MB/s eta 0:00:00
  Attempting uninstall: dnspython
    Found existing installation: dnspython 2.3.0
    Uninstalling dnspython-2.3.0:
      Successfully uninstalle

In [ ]:
# Enter your credentials
user='*********'
password='********'

classclient = MongoClient("mongodb+srv://******:********.pd0k4ky.mongodb.net/?retryWrites=true&w=majority")
ClassDb = classclient.ClassDb

In [ ]:
# If this runs successfully, step 1: deploy MongoDB is now finished
ClassDb

Database(MongoClient(host=['ac-q0g0fnx-shard-00-01.pd0k4ky.mongodb.net:27017', 'ac-q0g0fnx-shard-00-00.pd0k4ky.mongodb.net:27017', 'ac-q0g0fnx-shard-00-02.pd0k4ky.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-33tsic-shard-0', ssl=True, retrywrites=True, w='majority'), 'ClassDb')

In [ ]:
#Create a collection if it doesn't exists
#Indices help in maintaining the database optimally...it is typically what we search for.
#Indices can also be unique. In this case, there might be two people with the same name, so we use name+email as the index.

# in my dictionary, there must be a key named "ETF"(eg. QQQ, FPY...all tickers as values) as the index
ClassDb["Project"].create_index([("ETFs", pymongo.ASCENDING)],unique=True)

### Step 2: Load the ETF master list

In [ ]:
# Get the ETF master list (universe) 
import pandas as pd
ETFs_universe = pd.read_excel('https://www.dropbox.com/s/1a4u95oj30x68k8/ETF1.xlsx?raw=1',skiprows=[1]) # we  want to drop the first row(the row with index 1)
ETFs_universe.to_csv()

",Name,Ticker,Class Assets (MLN USD),Fund Assets (MLN USD),Holdings,Primary,Cross\n0,SPDR S&P 500 ETF Trust,SPY US,363010.72,363010.72,505.0,Y,N\n1,iShares Core S&P 500 ETF,IVV US,293184.09,293184.09,507.0,Y,N\n2,Vanguard Total Stock Market ETF,VTI US,267576.91,267576.91,4065.0,Y,N\n3,Vanguard S&P 500 ETF,VOO US,267321.03,267321.03,510.0,Y,N\n4,Invesco QQQ Trust Series 1,QQQ US,146212.17,146212.17,103.0,Y,N\n5,Vanguard Value ETF,VTV US,99625.23,99625.23,351.0,Y,N\n6,Vanguard Growth ETF,VUG US,71103.72,71103.72,257.0,Y,N\n7,iShares Core S&P Small-Cap ETF,IJR US,67803.27,67803.27,610.0,Y,N\n8,iShares Core S&P Mid-Cap ETF,IJH US,66049.36,66049.36,402.0,Y,N\n9,Vanguard Dividend Appreciation ETF,VIG US,64664.83,64664.83,293.0,Y,N\n10,iShares Russell 1000 Growth ETF,IWF US,58673.29,58673.29,518.0,Y,N\n11,iShares Russell 2000 ETF,IWM US,53444,53444,1951.0,Y,N\n12,iShares Russell 1000 Value ETF,IWD US,53037.79,53037.79,860.0,Y,N\n13,Vanguard Mid-Cap ETF,VO US,51306.98,51306.98,362.0,Y,N\n14,Va

In [ ]:
ETFs_universe.head(5) # Checking the first 5 rows of the dataset

,Name,Ticker,Class Assets (MLN USD),Fund Assets (MLN USD),Holdings,Primary,Cross
0,SPDR S&P 500 ETF Trust,SPY US,363010.72,363010.72,505.0,Y,N
1,iShares Core S&P 500 ETF,IVV US,293184.09,293184.09,507.0,Y,N
2,Vanguard Total Stock Market ETF,VTI US,267576.91,267576.91,4065.0,Y,N
3,Vanguard S&P 500 ETF,VOO US,267321.03,267321.03,510.0,Y,N
4,Invesco QQQ Trust Series 1,QQQ US,146212.17,146212.17,103.0,Y,N


In [ ]:
# For the "Ticker" column, we need to get rid of the " US"
ETFs_universe['Ticker'] = ETFs_universe['Ticker'].str.replace('US','')
ETFs_universe['Ticker']

0        SPY 
1        IVV 
2        VTI 
3        VOO 
4        QQQ 
        ...  
1389    PUNK 
1390    SAMT 
1391    SANE 
1392    STXV 
1393    TIME 
Name: Ticker, Length: 1394, dtype: object

### Step 3: Web Mining

- Retrive data from 3 sources: iShares, Investco, and StockAnalysis until we have all information we need for every ETFs in the ETFs_universe
- Push the updated dataset to MongoDb after every loop

### Web Parsing - StockAnalysis

In [ ]:
def getSAHoldings(etf):
  #Get holdings
  url = f'https://stockanalysis.com/etf/{etf}/holdings/'
  try:
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    req = Request(url=url,headers=headers) 
    resp = urlopen(req)    
  except:
    raise Exception(f'Error for {etf}')
  html = BeautifulSoup(resp, features="lxml")
  holdings={}
  for row in html.find('table').find_all('tr'):
    # cell is stock name
    # remove numbers
    # if cell is entirely made of numbers, remove
    cells=[d.text for d in row.find_all('td')]
    if len(cells)<4:
      continue
    wgt=float(cells[3][:-1]) #Remove % sign
    holdings[(cells[1].strip()).replace('.','')]=wgt*0.01
  return holdings

In [ ]:
# Checking whether the function is working or not

print(getSAHoldings('ARKK'))
print(getSAHoldings('IVV'))
print(getSAHoldings('SPY'))

{'TSLA': 0.1077, 'ZM': 0.0834, 'EXAS': 0.07540000000000001, 'ROKU': 0.0649, 'SQ': 0.0576, 'PATH': 0.0535, 'SHOP': 0.0527, 'COIN': 0.0455, 'TDOC': 0.0455, 'U': 0.0417, 'TWLO': 0.0394, 'CRSP': 0.0358, 'NTLA': 0.0354, 'DKNG': 0.0351, 'BEAM': 0.033100000000000004, 'RBLX': 0.029900000000000003, 'PD': 0.029900000000000003, 'HOOD': 0.025400000000000002, 'DNA': 0.0207, 'PACB': 0.0184, 'TXG': 0.0174, 'VCYT': 0.015300000000000001, 'TWST': 0.0123, 'TWOU': 0.0084, 'NVTA': 0.005, 'CERS': 0.0043, 'VERV': 0.004, 'n/a': 0.0023, 'MTLS': 0.0}
{'AAPL': 0.06570000000000001, 'MSFT': 0.0575, 'AMZN': 0.0257, 'GOOGL': 0.0172, 'BRKB': 0.0161, 'NVDA': 0.016, 'TSLA': 0.015600000000000001, 'GOOG': 0.015300000000000001, 'XOM': 0.013600000000000001, 'UNH': 0.0131, 'JNJ': 0.0124, 'JPM': 0.0121, 'META': 0.011899999999999999, 'V': 0.010900000000000002, 'HD': 0.0095, 'PG': 0.0095, 'MA': 0.0091, 'CVX': 0.0088, 'MRK': 0.0078000000000000005, 'LLY': 0.0078000000000000005, 'ABBV': 0.0074, 'BAC': 0.0074, 'PFE': 0.00709999999

In [ ]:
def checkPresence(etfs , ticker): # Function which will ensure that no ETF is repeated
  
  result=etfs['ticker'].isin([ticker])
  if result.any():
      return True
  else:
      return False

In [ ]:
# Loop over all ETFs from StockAnalysis and push data to MongoDb
etfs = pd.DataFrame(list(ClassDb["Etf_List"].find()))
empty_etf_sa = [] # list of etfs that didn't get holding data from StockAnalysis
for i in ETFs_universe['Ticker']:
  if checkPresence(etfs , i.strip()): continue
  ticker_name = i.strip()  
  try:
    temp_dict = {}
    value = getSAHoldings(ticker_name) 
    temp_dict = {"ticker":ticker_name,
                "Holdings":value}
  # Pass result to MongoDB
    try:
      result = ClassDb["Etf_List"].insert_one(temp_dict)
    except:
      pass
  except:
    empty_etf_sa.append(ticker_name)
    pass

len(empty_etf_sa)

### Web Parsing - Investco ETF

Web Parsing - Investco ETF
- the links look like this: https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=ICLO

In [ ]:
def getInvestcoHoldings(etfname): 
  url=f'https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker={etfname}'
  investcodf=pd.read_csv(url, thousands=',')
  investcodf.fillna(0.0)
  investcodf['Holding Ticker']=investcodf['Holding Ticker'].str.strip()
  
  try:
    investcodf=investcodf.set_index('Holding Ticker')
    return (investcodf['MarketValue']/investcodf['MarketValue'].sum()).to_dict()     
  except:
    investcodf=investcodf.set_index(' Holding Ticker')
    return (investcodf[' MarketValue']/investcodf[' MarketValue'].sum()).to_dict()  

In [ ]:
print(getInvestcoHoldings("QQQ")) # Checking whether the function is working or not

{'MSFT': 0.12280723332345951, 'AAPL': 0.12148949996308142, 'AMZN': 0.06261252905768702, 'NVDA': 0.04267137559217043, 'TSLA': 0.03909296130087149, 'GOOG': 0.03630042217458524, 'GOOGL': 0.03620050333461153, 'META': 0.031943599811816076, 'PEP': 0.01980169121136001, 'AVGO': 0.019598802192509347, 'COST': 0.018051416628974026, 'CSCO': 0.015836914949628637, 'TMUS': 0.01458374224234119, 'ADBE': 0.014068908762419917, 'CMCSA': 0.013498717546864438, 'TXN': 0.013014047950699334, 'NFLX': 0.012609371299193288, 'QCOM': 0.011794968463689484, 'HON': 0.01099426234373464, 'AMD': 0.010716413880832871, 'AMGN': 0.010554751907053765, 'SBUX': 0.010026622901988459, 'INTU': 0.009476159297730708, 'INTC': 0.009358692841015414, 'GILD': 0.008870171724732782, 'AMAT': 0.008054203205255722, 'ADP': 0.007641269953871623, 'PYPL': 0.0075138829495370795, 'BKNG': 0.007431840272451136, 'ADI': 0.007414015208595015, 'MDLZ': 0.007296260835903524, 'ISRG': 0.0070700959620854245, 'REGN': 0.006617528115031362, 'VRTX': 0.00622700884

In [ ]:
# Loop over all ETFs from StockAnalysis and push data to MongoDb
etfs = pd.DataFrame(list(ClassDb["Etf_List"].find()))
empty_etf_inv = [] # list of etfs that didn't get holding data from Investco
for i in ETFs_universe['Ticker']:
  if checkPresence(etfs , i.strip()): continue
  ticker_name = i.strip()  
  try:
    temp_dict = {}
    value = getInvestcoHoldings(ticker_name) 
    temp_dict = {"ticker":ticker_name,
                 "Holdings":value}
    # Pass result to MongoDB
    try:
      result = ClassDb["Etf_List"].insert_one(temp_dict)
    except:
      pass
  except:
    empty_etf_inv.append(ticker_name)
    pass

len(empty_etf_inv)

### Web Parsing - iShares ETFs

In [ ]:
url='https://www.ishares.com/us/products/etf-investments#/?productView=etf&pageNumber=1&sortColumn=totalNetAssets&sortDirection=desc&dataView=keyFacts'
resp = urlopen(Request(url=url,headers={'user-agent': 'my-app/0.0.1'}))   

  * Notice that the data we want is in a table
    * There are \<tr>, table rows
    * Inside are \<td>, table columns/data
    * Inside are \<a>, anchors, and the text is the ETF ticker that we want.
    * and the value corresponding to "href" in the anchor \<a> tag corresponds to the link that we also want to capture.

In [ ]:
html = BeautifulSoup(resp, features="lxml") # it is quite hard to read html unless we use Beautiful Soup to parse the tree
#html
print(html.prettify())

<!DOCTYPE html>
<html lang="en" prefix="og: http://ogp.me/ns#" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   iShares ETF Investments List | iShares - BlackRock
  </title>
  <meta content="text/html;charset=utf-8" http-equiv="Content-type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="product list, product screener, ishares list, ishares product list" name="keywords"/>
  <meta content="Find the full list of iShares ETFs here. Use these low cost, tax efficient funds to strengthen the core of your portfolio." name="description"/>
  <meta content="iShares ETF Investments List | iShares - BlackRock" name="articleTitle"/>
  <meta content="Find the full list of iShares ETFs here. Use these low cost, tax efficient funds to strengthen the core of your portfolio." name="pageSummary"/>
  <meta content="iShares ETF Investments List | iShares - BlackRock" name="featureImageAltText"/>
  <meta content="iShares ETF Investments Lis

We will store everything we want to capture in dict *ticToURL*
* The key is ticker of the ETF
* The value is the full URL for more information of the ETF.
* Sometimes, the key is not a ticker but a fund name, etc., and so we will just skip those.
* By inspection, the CSV file we are interested in can be found by hacking the URL and adding the following at the end.
`'/1467271812596.ajax?fileType=csv&fileName=IWM_holdings&dataType=fund'`

In [ ]:
# Now we get all the href text 
# <a> + href makes the link clickable, so we are trying to find all <a> plus the href text
# INCLUDE IN PROJECT1a

ticToURL={}

for row in html.find_all('tr'):
  try:
    for data in row.find_all('a'):
      if len(data.text)>0 and len(data.text)<5:
        # print(f'Ticker: {data.text} -> Link {data["href"]}')
        ticToURL[data.text]='https://www.ishares.com/'+data['href']+'/1467271812596.ajax?fileType=csv&fileName=IWM_holdings&dataType=fund'
  except:
    0
#Then get the CSV from ticToURL mapped URL
def getiShareHoldings(etfname):
  df=pd.read_csv(ticToURL[etfname],skiprows=range(0,9), thousands=',') #Read CSV from URL we did in step 1
  df['Ticker']=df['Ticker'].str.strip()
  df=df[df['Asset Class']=='Equity'].set_index('Ticker')               #Select only Equity rows
  return (df["Market Value"]/df["Market Value"].sum()).to_dict()       # Convert to a dict

In [ ]:
# just to test it out
print(getiShareHoldings('IWM'))

{'INSP': 0.0029982074429190497, 'MTDR': 0.002949595701532111, 'SAIA': 0.0029150163102145046, 'IRDM': 0.0028742747479342575, 'CROX': 0.00282235800783288, 'EME': 0.0027875647051851305, 'TXRH': 0.0027352755481783804, 'RBC': 0.0027217413929317377, 'CMC': 0.0026373023535997456, 'HALO': 0.0026277012180995875, 'SWAV': 0.0025918711521816028, 'CHX': 0.0025711955830169295, 'MUR': 0.002569894799863936, 'ADC': 0.002565465911365154, 'STAG': 0.0025445500396300623, 'KNSL': 0.00253330643717674, 'LNW': 0.0024634109714179652, 'SLAB': 0.002459021400807285, 'SIGI': 0.0023956999404777923, 'SSB': 0.0023863190721516957, 'ATKR': 0.0023789421657560476, 'CHRD': 0.002368852482125693, 'AQUA': 0.0023630255597801874, 'GTLS': 0.0023586285803121414, 'MUSA': 0.0023140955684995577, 'NOVT': 0.002307402156792079, 'MEDP': 0.0022893149269951826, 'KRTX': 0.002273270651308883, 'UFPI': 0.0022706969103385537, 'EXLS': 0.002264273733834923, 'UBSI': 0.002195035780928407, 'AIT': 0.0021775149447331612, 'CELH': 0.0021774464926863106

In [ ]:
# Apply this function to all ETFs that we got from iShares

etfs = pd.DataFrame(list(ClassDb["Etf_List"].find()))
empty_etf = [] 
for i in ETFs_universe['Ticker']:
  if checkPresence(etfs , i.strip()): continue
  ticker_name = i.strip()  
  try:
    temp_dict = {}
    value = getiShareHoldings(ticker_name) 
    temp_dict = {"ticker":ticker_name,
                 "Holdings":value}
    # Pass result to MongoDB
    try:
      result = ClassDb["Etf_List"].insert_one(temp_dict)
    except:
      pass
  except:
    empty_etf.append(ticker_name)
    pass

len(empty_etf)

### Final Step: Cosine Similarity & Recommendation

In [ ]:
etfs = pd.DataFrame(list(ClassDb["Etf_List"].find())) #Retireving data from mongoDB
etfs

,_id,ticker,Holdings
0,63e6b83e8263db040286ad0a,SPY,"{'AAPL': 0.066, 'MSFT': 0.0575, 'AMZN': 0.0255..."
1,63e6b83e8263db040286ad0b,IVV,"{'AAPL': 0.06620000000000001, 'MSFT': 0.0571, ..."
2,63e6b83f8263db040286ad0c,VTI,"{'AAPL': 0.053700000000000005, 'MSFT': 0.0455,..."
3,63e6b83f8263db040286ad0d,VOO,"{'AAPL': 0.0631, 'MSFT': 0.054000000000000006,..."
4,63e6b83f8263db040286ad0e,QQQ,"{'MSFT': 0.1222, 'AAPL': 0.12050000000000001, ..."
...,...,...,...
183,63e6c60da740be09cad73c64,XLV,"{'UNH': 0.09210000000000002, 'JNJ': 0.08560000..."
184,63e6c60da740be09cad73c65,XLY,"{'AMZN': 0.2346, 'TSLA': 0.153, 'HD': 0.0888, ..."
185,63e6c60da740be09cad73c66,XME,"{'CLF': 0.0525, 'UEC': 0.047400000000000005, '..."
186,63e6c664bb75c700cdb2c790,XOP,"{'PBF': 0.025400000000000002, 'KOS': 0.0252, '..."


#### Converting the above data into a DataFrame

In [ ]:
dff= pd.DataFrame()
dff.insert(0,'stock tickers',0)

lists=[]

#get stock tickers to df
for eft in range(0,len(etfs['Holdings'])):
  head= list(etfs['Holdings'][eft])
  for i in head:
    lists.append(i.strip()) 

key = pd.DataFrame(lists)
key = key.drop_duplicates()
dff['stock tickers'] = key
dff=dff.set_index('stock tickers')
# df['EFT']['AAPL']=5
dff

""
stock tickers
AAPL
MSFT
AMZN
GOOGL
NVDA
...
APE
KIND
SST


In [ ]:
# get 
new_etfs=etfs[['ticker', 'Holdings']]
for ku in range(0,len(new_etfs['ticker'])):
  col=(new_etfs['ticker'][ku]).strip()
  dff.insert(ku,col,0.000)
  data= new_etfs['Holdings'][ku]
  value =pd.DataFrame(data ,index=[0])
  value = pd.DataFrame(value.iloc[0])
  for ob in list(data):
    dff[col][ob] = value[0][ob] 

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
dff_col = dff.columns 

In [ ]:
import scipy
from scipy import spatial

user_recc="QQQ"

cosineds= pd.DataFrame()
cosineds.insert(0,"ticker",0.000)
cosineds["ticker"]=list(dff_col)
cosineds.insert(1,"Cosine_DIS",0.000)

values=[]
for vh in range(0,len(dff_col)):
  values.append(1.0-scipy.spatial.distance.cosine(dff[dff_col[vh]].fillna(0),dff[user_recc].fillna(0)))
cosineds["Cosine_DIS"]=values
cosineds=cosineds.sort_values(by="Cosine_DIS" ,ascending=False).iloc[1:].reset_index(drop=True)

cosineds

,ticker,Cosine_DIS
0,QQQM,0.999984
1,ONEQ,0.939253
2,QYLD,0.935634
3,SCHG,0.930735
4,IWF,0.925692
...,...,...
182,IJS,0.000000
183,SPSM,0.000000
184,CHIE,-0.000089
185,AMLP,-0.000652


In [ ]:
cosineds.head(10)

,ticker,Cosine_DIS
0,QQQM,0.999984
1,ONEQ,0.939253
2,QYLD,0.935634
3,SCHG,0.930735
4,IWF,0.925692
5,VUG,0.925330
6,MGK,0.924978
7,IWY,0.924556
8,VONG,0.920500
9,ESGV,0.897405


##### End of the code